# Load Data

In [1]:
!pip install rectools[all] -q

zsh:1: no matches found: rectools[all]


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install polars==1.25.2 >> _


In [3]:
# !pip install implicit >> _

In [5]:
# # takes 5 minutes
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/clickstream.pq -O data/clickstream.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/test_users.pq -O data/test_users.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/cat_features.pq -O data/cat_features.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/text_features.pq -O data/text_features.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/events.pq -O data/events.pq >> _


In [1]:
from datetime import timedelta
import polars as pl
import implicit

In [7]:
DATA_DIR = 'data/'

df_test_users = pl.scan_parquet(f'{DATA_DIR}/test_users.pq')
df_clickstream = pl.read_parquet(f'{DATA_DIR}/clickstream.pq')

df_cat_features = pl.read_parquet(f'{DATA_DIR}/cat_features.pq')
df_text_features = pl.read_parquet(f'{DATA_DIR}/text_features.pq')
df_event = pl.read_parquet(f'{DATA_DIR}/events.pq')

# PREPARE TRAIN EVAL

In [ ]:
EVAL_DAYS_TRESHOLD = 14

treshhold = df_clickstream['event_date'].max() - timedelta(days=EVAL_DAYS_TRESHOLD)

In [10]:
df_train = df_clickstream.filter(df_clickstream['event_date']<= treshhold)
df_eval = df_clickstream.filter(df_clickstream['event_date']> treshhold)[['cookie', 'node', 'event']]

In [11]:
df_eval = df_eval.join(df_train, on=['cookie', 'node'], how='anti')


In [12]:
df_eval = df_eval.filter(
    pl.col('event').is_in(
        df_event.filter(pl.col('is_contact')==1)['event'].unique()
    )
)

In [13]:
df_eval = df_eval.filter(
        pl.col('cookie').is_in(df_train['cookie'].unique())
    ).filter(
        pl.col('node').is_in(df_train['node'].unique())
    )

In [14]:
df_eval = df_eval.unique(['cookie', 'node'])

# TRAIN MODEL

## ALS

In [15]:
def get_als_pred(users, nodes, user_to_pred):
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()
        
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v:k for k,v in item_id_to_index.items()}
    
    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()
    
    values = [1] * len(users)
    
    sparse_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(item_ids)))
    
    model = implicit.als.AlternatingLeastSquares(iterations=10, factors=60)
    model.fit(sparse_matrix, )
    
    
    user4pred = np.array([user_id_to_index[i] for i in user_to_pred])
    
    recommendations, scores = model.recommend(user4pred, sparse_matrix[user4pred], N=40, filter_already_liked_items=True)
    
    df_pred = pl.DataFrame(
        {
            'node': [
                [index_to_item_id[i] for i in i] for i in recommendations.tolist()
            ], 
             'cookie': list(user_to_pred),
            'scores': scores.tolist()
            
        }
    )
    df_pred = df_pred.explode(['node', 'scores'])
    return df_pred

In [16]:
from scipy.sparse import csr_matrix
import numpy as np
import implicit


users = df_train["cookie"]
nodes = df_train["node"]
eval_users = df_eval['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes,eval_users )




/Users/admastryukov/miniconda3/lib/python3.12/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 10 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

## popular

In [17]:
def get_popular(df):
    popukar_node = df.group_by('node').agg(pl.col('cookie').count()).sort('cookie').tail(40)['node'].to_list()
    df_pred_pop = pl.DataFrame({'node': [popukar_node for i in range(len(eval_users))], 'cookie': eval_users})
    df_pred_pop = df_pred_pop.explode('node')
    return df_pred_pop

train_pop = get_popular(df_train)


# CALC EVAL METRICS

In [18]:
def recall_at(df_true, df_pred, k=40):
    return  df_true[['node', 'cookie']].join(
        df_pred.group_by('cookie').head(k).with_columns(value=1)[['node', 'cookie', 'value']], 
        how='left',
        on = ['cookie', 'node']
    ).select(
        [pl.col('value').fill_null(0), 'cookie']
    ).group_by(
        'cookie'
    ).agg(
        [
            pl.col('value').sum()/pl.col(
                'value'
            ).count()
        ]
    )['value'].mean()


In [19]:
recall_at(df_eval, df_pred, k=40)

0.15041722015925818

In [20]:
recall_at(df_eval, train_pop, k=40)

0.05806730855297021

# SUMBIT

In [21]:
users = df_clickstream["cookie"]
nodes = df_clickstream["node"]
test_users = df_test_users['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes, test_users )


  0%|          | 0/10 [00:00<?, ?it/s]

In [22]:
df_pred.write_csv('prediction.csv')